## Simulation using Gaussian distributed volumes

In [1]:
import numpy as np
import nibabel as nib
import os
import matplotlib.pyplot as plt
import math

%cd utils
from sim_functions import optimize_measurement
%cd ..

c:\Users\User\msc_project\Image-processing-strategies\MRsim\utils
c:\Users\User\msc_project\Image-processing-strategies\MRsim


c:\Users\User\miniconda3\envs\mrsim\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


## Simulating for QSM </br>
Last echo time should be at least close to T2s value of tissue of interest. </br>
For SC, we want inbetween GM and WM: 59.5 ms </br>
The first echo time should be the shortest possible, and echo spacing should be uniform. </br>
Flip angle should be Ernst Angle $(cos^(exp(-TR/T1))-1)$ for tissue in the target region  </br>


In [2]:
# Echo time list creation
# Change number of echoes, start echo and last echo
num_echoes = 5
start_echo = 0.002
last_echo = 0.058

############## ### ########## ####### # # ##### # #  #### # ## ### 
echo_diff = last_echo - start_echo 
echo_dist = echo_diff / (num_echoes-1)
custom_TE_list = np.zeros(num_echoes)
custom_TE_list[0] = start_echo
custom_TE_list[num_echoes-1] = last_echo

for i in range(num_echoes-2):
    custom_TE_list[i+1] = custom_TE_list[i] + echo_dist

print(custom_TE_list)
print(echo_dist)

[0.002 0.016 0.03  0.044 0.058]
0.014


In [3]:
# Calculating Erst Angle
TR_val = 6 # in miliseconds
T1_val = 854 # in miliseconds
fa_ernst = math.acos(np.exp(-TR_val/T1_val))
# acos returns in radians
fa_ern_deg = math.degrees(fa_ernst)

#flip_ang = 24
#te_secs = [8, 9.5] # Using it in seconds to make it easier to run with texture on T2s values ??

# The last requirement for the code is the dimensions, which can be easily acquired from the volume you work with
# The dimensions needed because we will create a 4D array, where we will store the echo time data and we want to maintain the initial 3D dimensions.
data = nib.load("data/ratatouille_crop.nii.gz")
np_data = data.get_fdata()
dimensions = np.array(np_data.shape)
dimensions
math.degrees(fa_ernst)
# In QSM RC 2.0 the flip angle is very low

6.783840992457034

In [4]:
# Loading the data, Gaussian distributed volumes
g_pd= nib.load("output/gauss_low_sd_pd_dist.nii.gz")
gauss_pd_vol = g_pd.get_fdata()
g_t2s = nib.load("output/gauss_low_sd_t2s_dist.nii.gz")
gauss_t2s_vol = g_t2s.get_fdata()
#deltaB0 = nib.load("data/fieldmap_crop.nii.gz")
pro_deltaB0 = nib.load("output/fm_05_sd.nii.gz")
dB0 = pro_deltaB0.get_fdata()
fieldStrength = 3
# suppress warnings due to Overflow on exponential while iterating
#import warnings
#warnings.filterwarnings('ignore') 

In [4]:
# I think the problem is when we have T2s value sooooooooo low, like in the lungs and air where we have 0.01
decay_gauss = np.zeros(dimensions)
for i in range(dimensions[0]):
    for j in range(dimensions[1]):
        for k in range(dimensions[2]):
            with np.errstate(over='raise'):
                try:
                    decay_gauss[i, j, k] = np.exp(-0.008 / gauss_t2s_vol[i, j, k])
                except FloatingPointError:
                    print(f"Overflow detected at index ({i}, {j}, {k}) with T2* value {gauss_t2s_vol[i, j, k]}")
                    decay_gauss[i, j, k] = np.finfo(np.float64).max  # Assign max value or another placeholder

In [30]:
#An example of what a values can rise a overflow in exponential
# This value was in the T2s volume because the gaussian distribution centered at 0.01
# Which was 'air' label created a few pixels with this offlayer values that caused the issue 
# returning infinity and hence ITK could not open the image
#np.exp(-0.008/-4.346339950507538e-06)

C:\Users\User\AppData\Local\Temp\ipykernel_83356\1727653921.py:1: RuntimeWarning: overflow encountered in exp
  np.exp(-0.008/-4.346339950507538e-06)


inf

In [5]:
# For gaussian data
gauss_magnitude, gauss_phase = optimize_measurement(gauss_pd_vol, gauss_t2s_vol, dimensions, dB0, fa_ern_deg, custom_TE_list, fieldStrength)
# By default, B0 should be 3 Tesla

Starting optimize_measurement
Processing TE[0] = 0.002
Starting optimized_signal
sin:  0.11812391963869848
handedness=left
Coefficient of phase factor:  -1.6051331246399998j
Finished optimized_signal
mag shape: (109, 129, 218), phase_arr shape: (109, 129, 218)
Processing TE[1] = 0.016
Starting optimized_signal
sin:  0.11812391963869848
handedness=left
Coefficient of phase factor:  -12.841064997119998j
Finished optimized_signal
mag shape: (109, 129, 218), phase_arr shape: (109, 129, 218)
Processing TE[2] = 0.03
Starting optimized_signal
sin:  0.11812391963869848
handedness=left
Coefficient of phase factor:  -24.076996869599995j
Finished optimized_signal
mag shape: (109, 129, 218), phase_arr shape: (109, 129, 218)
Processing TE[3] = 0.044
Starting optimized_signal
sin:  0.11812391963869848
handedness=left
Coefficient of phase factor:  -35.31292874207999j
Finished optimized_signal
mag shape: (109, 129, 218), phase_arr shape: (109, 129, 218)
Processing TE[4] = 0.058
Starting optimized_sign

In [7]:
gauss_phase.min()

-3.141592653589793

In [6]:
# Save the magnitude and the phase to analyze them
# Remember to change the name, if not it will overwrite it!
# This version for gaussian distributed volumes
temp_mag_g = nib.Nifti1Image(gauss_magnitude, affine=data.affine)
temp_ph_g = nib.Nifti1Image(gauss_phase, affine=data.affine)
path_mag_g = os.path.join('output','qsm_trials_phantom5_mag.nii.gz')
path_ph_g = os.path.join('output','qsm_trials_phantom5_ph.nii.gz')
nib.save(temp_mag_g, path_mag_g)
nib.save(temp_ph_g, path_ph_g)

del temp_mag_g, temp_ph_g

## For QSM trials </br>
We need to save each echo separately for processing with Shimming Toolbox to get the Fieldmap


In [7]:
# Automating saving every echo
out_dir = 'echoes'
os.makedirs(out_dir,exist_ok=True)

for i in range(num_echoes):
    # Create the nifti file according to the echo
    temp_echo_mag = nib.Nifti1Image(gauss_magnitude[..., i], affine = data.affine)
    temp_echo_ph = nib.Nifti1Image(gauss_phase[..., i], affine = data.affine)

    # Create the path to save this files
    path_mag = os.path.join('echoes',f'gauss_sim_mag_echo{i+1}.nii.gz')
    path_ph = os.path.join('echoes',f'gauss_sim_ph_echo{i+1}.nii.gz')

    nib.save(temp_echo_mag, path_mag)
    nib.save(temp_echo_ph, path_ph)


In [ ]:
# For the first step of the QSM trials we use Shimming Toolbox.
# To acquire the fieldmaps we need a .json sidecar for every echo
# The only information necessary is: 
{
	"Modality": "MR",
	"MagneticFieldStrength": 3,
	"Manufacturer": "Siemens",
	"EchoNumber": 1,
	"EchoTime": 0.008,
	"FlipAngle": 24
}
# Manually edit the echo number, flip angle and the echo time appropriately